# Process
- split text into chunks
- prompt the LLM to extract the classes (for the whole chunk)
- then split the text sentence by sentence and prompt the LLM to extract individuals and relations 

In [15]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

AZURE_DEPLOYMENT_GPT41 = os.getenv("AZURE_DEPLOYMENT_GPT41")
AZURE_DEPLOYMENT_GPT41_NANO = os.getenv("AZURE_DEPLOYMENT_GPT41_NANO")

gpt41_nano = AzureChatOpenAI(
    azure_deployment=AZURE_DEPLOYMENT_GPT41_NANO,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    temperature=0.3
)

In [2]:
TTL_HEADER = """@prefix : <http://example.org/onto#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

"""

In [ ]:
import nltk
from nltk import sent_tokenize

nltk.download('punkt')

def read_text_file(file: str) -> list:
    """
    Read a text file and return a list of sentences.
    """
    with open(file, 'r') as file:
        text = file.read()
        sentences = nltk.sent_tokenize(text, language='portuguese')
    return sentences


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tiago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
prompt_c_step_1 = """Extract classes in ttl format from the following text, only return the created ttl code. Make sure to label all classes as of type rdfs:Class: """


prompt_c_step_2 = """Extract individuals and relations in ttl format from the following sentence, based on the given classes. 
Only return the ttl code, include the classes, individuals, and relations in this sentence. 
Make sure to label all classes as of rdfs:Class, all individuals as owl:NamedIndividual, and all properties as owl:ObjectProperty, owl:DatatypeProperty, or owl:AnnotationProperty: """


In [12]:
text = read_text_file('../../data/texts/application_example.txt')
display(text)

['Vouchers para Startups – Novos produtos\ndigitais/tecnológicos Granter.ai\nAtividade Económica Principal do Beneficiário (Fundamentação)\u200b\n(limite XX)\n\nA atividade principal da Granter.ai enquadra-se plenamente nos setores intensivos em\ntecnologia, uma vez que a empresa se dedica ao desenvolvimento de soluções baseadas em\ninteligência artificial (IA), aplicadas à área das candidaturas a fundos públicos.',
 'O projeto\nproposto, que passa pela criação de um agente de IA que vai apoiar as empresas na\nelaboração de candidaturas a programas de financiamento, integra atividades de\ninvestigação e desenvolvimento tecnológico, incorporando metodologias avançadas como\nRedes Generativas Adversariais (GANs) e Adversarial In-Context Learning (Adv-ICL),\npermitindo gerar conteúdos especializados e adaptados ao contexto dos clientes.',
 'Trata-se\nde um software proprietário, desenvolvido internamente, que valoriza a aplicação prática de\nconhecimento técnico e científico, ao transform

In [23]:
print(prompt_c_step_1 + '\n\n' + text[1])

Extract classes in ttl format from the following text, only return the created ttl code. Make sure to label all classes as of type rdfs:Class: 

O projeto
proposto, que passa pela criação de um agente de IA que vai apoiar as empresas na
elaboração de candidaturas a programas de financiamento, integra atividades de
investigação e desenvolvimento tecnológico, incorporando metodologias avançadas como
Redes Generativas Adversariais (GANs) e Adversarial In-Context Learning (Adv-ICL),
permitindo gerar conteúdos especializados e adaptados ao contexto dos clientes.


In [25]:
def get_azure_response(text, prompt, ontology_info=None):
    """Get response from Azure OpenAI"""
    if ontology_info is None:
        message_content = f"{prompt} {text}"
    else:
        # Convert ontology_info list to string if needed
        if isinstance(ontology_info, list):
            ontology_info = "\n".join(ontology_info)
        message_content = f"{prompt} {text}, {ontology_info}"

    print(f"Message: {message_content}")

    response = gpt41_nano.invoke([message_content])
    
    return response.content

In [26]:
response = get_azure_response(text[0], prompt_c_step_1)
print(response)

Message: Extract classes in ttl format from the following text, only return the created ttl code. Make sure to label all classes as of type rdfs:Class:  Vouchers para Startups – Novos produtos
digitais/tecnológicos Granter.ai
Atividade Económica Principal do Beneficiário (Fundamentação)​
(limite XX)

A atividade principal da Granter.ai enquadra-se plenamente nos setores intensivos em
tecnologia, uma vez que a empresa se dedica ao desenvolvimento de soluções baseadas em
inteligência artificial (IA), aplicadas à área das candidaturas a fundos públicos.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

:VouchersParaStartupsNovosProdutosDigitalTecnologicos a rdfs:Class .

:GranterAI a rdfs:Class .

:AtividadeEconomicaPrincipalDoBeneficiario a rdfs:Class .


In [22]:
from typing import List
def get_results_from_response(response_content: str) -> List[str]:
    """Extract results from Azure OpenAI response content"""
    try:
        return response_content.split('\n')
    except Exception as e:
        print(f'Error processing response: {e}')
        return []
    
get_results_from_response(response)

['```ttl',
 '@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .',
 '',
 ':VouchersParaStartupsNovosProdutosDigitalTecnologicos a rdfs:Class .',
 '',
 ':GranterAI a rdfs:Class .',
 '',
 ':AtividadeEconomicaPrincipalDoBeneficiario a rdfs:Class .',
 '```']

In [31]:
def query_azure_gpt(text: list):
    """Query Azure GPT with the text"""
    relations = []

    try:
        # Step 1: Extract classes
        response_1 = get_azure_response(text, prompt_c_step_1)
        
        # Step 2: Extract relations for each sentence
        for sentence in text:
            response_2 = get_azure_response(sentence, prompt_c_step_2, get_results_from_response(response_1))
            relations.append(get_results_from_response(response_2))

    except Exception as e:
        print(f"Error querying Azure GPT: {e}")
        return []

    print(f"Extracted {len(relations)} relation sets")
    return relations

In [34]:
relations = query_azure_gpt(text[:10])
relations

Message: Extract classes in ttl format from the following text, only return the created ttl code. Make sure to label all classes as of type rdfs:Class:  ['Vouchers para Startups – Novos produtos\ndigitais/tecnológicos Granter.ai\nAtividade Económica Principal do Beneficiário (Fundamentação)\u200b\n(limite XX)\n\nA atividade principal da Granter.ai enquadra-se plenamente nos setores intensivos em\ntecnologia, uma vez que a empresa se dedica ao desenvolvimento de soluções baseadas em\ninteligência artificial (IA), aplicadas à área das candidaturas a fundos públicos.', 'O projeto\nproposto, que passa pela criação de um agente de IA que vai apoiar as empresas na\nelaboração de candidaturas a programas de financiamento, integra atividades de\ninvestigação e desenvolvimento tecnológico, incorporando metodologias avançadas como\nRedes Generativas Adversariais (GANs) e Adversarial In-Context Learning (Adv-ICL),\npermitindo gerar conteúdos especializados e adaptados ao contexto dos clientes.', 

[['@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .',
  '',
  ':VouchersParaStartups a rdfs:Class .',
  '',
  ':AtividadeEconómicaPrincipalBeneficiário a rdfs:Class .',
  '',
  ':Projeto a rdfs:Class .',
  '',
  ':SoftwareProprietário a rdfs:Class .',
  '',
  ':SoluçãoTecnológica a rdfs:Class .',
  '',
  ':FerramentaInteligente a rdfs:Class .',
  '',
  ':AgenteDeIA a rdfs:Class .',
  '',
  ':MetodologiaDeIA a rdfs:Class .',
  '',
  ':FundosPublicos a rdfs:Class .',
  '',
  ':Candidatura a rdfs:Class .',
  '',
  ':TecnologiaAvançada a rdfs:Class .',
  '',
  ':AutomaçãoDeProcessos a rdfs:Class .',
  '',
  ':SetorDeAltaTecnologia a rdfs:Class .',
  '',
  ':Inovação a rdfs:Class .',
  '',
  ':Crescimento a rdfs:Class .',
  '',
  ':FinanciamentoPublico a rdfs:Class .',
  '',
  ':GranTerAI a owl:NamedIndividual .',
  '',
  ':atividadePrincipalGranTerAI a owl:NamedIndividual .',
  '',
  ':setorTecnologia a owl:NamedIndividual .',
  '',
  ':desenvolvimentoIA a owl:NamedIndividual .',
  

In [35]:
def clean_relations(relations, ttl_output):
    for relation in relations:
        for triple in relation:
            if triple:
                cleaned_triple = triple.strip().strip('```').strip("ttl").strip("```")
                ttl_output.append(cleaned_triple)

                

def gpt_results_to_ttl(relations: List, output: str):
    ttl_output = []
    clean_relations(relations, ttl_output)

    with open(output, 'w') as f:
        f.write("\n".join(ttl_output))

In [37]:
gpt_results_to_ttl(relations, '../../output/ontologies/text/initial_test.ttl')